In [4]:
from preprocessing import preprocessing_csr
import collections

In [163]:
class treeNode:
    """variables:
    name of the node, a count
    nodelink used to link similar items
    parent vaiable used to refer to the parent of the node in the tree
    node contains an empty dictionary for the children in the node"""
    
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode      #needs to be updated
        self.children = []
        
    def get_child(self, name):
        
        for c in self.children:
            if c.name == name:
                return c
        
        return None
        
    
#increments the count variable with a given amount    
    def inc(self, numOccur):
        self.count += numOccur
#
#display tree in text. Useful for debugging        
    def disp(self, ind=1):
        print ('  '*ind, self.name, ' ', self.count)
        for child in self.children:
            child.disp(ind+1)

In [99]:
# testing the tree
rootNode = treeNode('pyramid',9,None)
rootNode.children.append(treeNode('eye',13,None))
rootNode.disp()

('  ', 'pyramid', ' ', 9)
('    ', 'eye', ' ', 13)


In [8]:
def updateHeader(nodeToTest, targetNode):   #this version does not use recursion
    while (nodeToTest.nodeLink != None):    #Do not use recursion to traverse a linked list!
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [9]:
def ascendTree(leafNode, prefixPath): #ascends from leaf node to root
    """ which ascends the tree, collecting the names of items it encounters"""

    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

In [44]:
def findPrefixPath(basePat, treeNode): #treeNode comes from header table
    """function iterates through the linked list until it hits the end. 
    For each item it encounters, it calls ascendTree().This list is 
    returned and added to the conditional pattern base dictionary 
    called condPats."""

    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1: 
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [108]:
def createTree(dataSet, minSup=1): #create FP-tree from dataset but don't mine
    """takes the dataset and the minimum support as arguments and builds the 
    FP-tree. This makes two passes through the dataset. The first pass goes 
    through everything in the dataset and counts the frequency of each term. 
    These are stored in the header table."""
    
    headerTable = {}

    #go over dataSet twice
    for trans in dataSet:   #first pass counts frequency of occurance
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]

#     for k in list(headerTable):  #remove items not meeting minSup
#         if headerTable[k] < minSup: 
#             del(headerTable[k])

    freqItemSet = list(headerTable.keys())
#     print 'freqItemSet: ',freqItemSet

    if len(freqItemSet) == 0: 
        return None, None  #if no items meet min support -->get out
    
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] #reformat headerTable to use Node link 
    
    print 'headerTable: ',headerTable

    retTree = treeNode('Null Set', 1, None) #create tree
    
    # dataSet.items() returns key-value tuple
    print dataSet.items()
    for tranSet, count in dataSet.items():  #go through dataset 2nd time
        localD = collections.OrderedDict()
        for item in tranSet:  #put transaction items in order
#             print item
#             if item in freqItemSet:
            localD[item] = headerTable[item][0]
#             t = item, headerTable[item][0]
#             localD.append(t)
        print 'localD: ',localD
# SHOULD STUDY THIS PART!
        if len(localD) > 0:
#             orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
#             print orderedItems
            items = list(v[0] for v in localD.items())
            print 'items: ', items
            updateTree(items, retTree, headerTable, count)   #populate tree with ordered freq itemset
            
            

    return retTree, headerTable #return tree and header table

In [180]:
def updateTree(items, inTree, headerTable, count):
    """grow the Fp-tree with an itemset"""

    if inTree.get_child(items[0]) != None: #check if orderedItems[0] in retTree.children
#         inTree.children[items[0]].inc(count) #incrament count
        inTree.get_child(items[0]).inc(count)
    else:   #add items[0] to inTree.children
        inTree.children.append(treeNode(items[0], count, inTree))
        
        if headerTable[items[0]][1] == None: #update header table 
            headerTable[items[0]][1] = inTree.children[len(inTree.children)-1] #W A R N I N G!
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[len(inTree.children)-1])
            
    if len(items) > 1: #call updateTree() with remaining ordered items
        updateTree(items[1::], inTree.get_child(items[0]), headerTable, count)

In [181]:
#The FP-tree
myFPtree, myHeaderTab = createTree(initSet, 3)
myFPtree.disp()

headerTable:  {'pr': [7, None], 'paano': [1, None], 'nn': [7, None], 'cc': [4, None], 'ano': [2, None], 'vb': [8, None], 'cd': [1, None], 'abbreviation': [7, None], 'saan': [4, None], 'rb': [2, None], 'dt': [4, None]}
[(('paano', 'pr', 'vb', 'dt', 'dt', 'nn', 'cc', 'pr', 'abbreviation'), 1), (('ano', 'dt', 'nn', 'vb', 'cc', 'nn', 'pr', 'vb', 'nn', 'abbreviation'), 1), (('saan', 'pr', 'vb', 'abbreviation'), 1), (('saan', 'pr', 'rb', 'vb', 'abbreviation'), 1), (('ano', 'dt', 'vb', 'vb', 'cc', 'nn', 'abbreviation'), 1), (('saan', 'pr', 'rb', 'nn', 'cc', 'nn', 'abbreviation'), 1), (('saan', 'pr', 'cd', 'vb', 'abbreviation'), 1)]
localD:  OrderedDict([('paano', 1), ('pr', 7), ('vb', 8), ('dt', 4), ('nn', 7), ('cc', 4), ('abbreviation', 7)])
items:  ['paano', 'pr', 'vb', 'dt', 'nn', 'cc', 'abbreviation']
localD:  OrderedDict([('ano', 2), ('dt', 4), ('nn', 7), ('vb', 8), ('cc', 4), ('pr', 7), ('abbreviation', 7)])
items:  ['ano', 'dt', 'nn', 'vb', 'cc', 'pr', 'abbreviation']
localD:  OrderedD

In [12]:
def loadSimpDat():
    simpDat = preprocessing_csr.format()
    return simpDat

In [13]:
def createInitSet(dataSet):
    """the createTree() function doesnt 
    take the input data as lists. It expects
    a dictionary with the itemsets as the 
    dictionary keys and the frequency as the
    value. A createInitSet() function does this
    conversion for you."""

    retDict = {}

    for trans in dataSet:
        retDict[tuple(trans)] = 1

    return retDict

In [14]:
#testing the sample data
simpDat = loadSimpDat()
print simpDat

[['saan', 'pr', 'vb', 'abbreviation'], ['paano', 'pr', 'vb', 'dt', 'dt', 'nn', 'cc', 'pr', 'abbreviation'], ['ano', 'dt', 'nn', 'vb', 'cc', 'nn', 'pr', 'vb', 'nn', 'abbreviation'], ['saan', 'pr', 'rb', 'nn', 'cc', 'nn', 'abbreviation'], ['saan', 'pr', 'vb', 'abbreviation'], ['saan', 'pr', 'cd', 'vb', 'abbreviation'], ['ano', 'dt', 'vb', 'vb', 'cc', 'nn', 'abbreviation'], ['saan', 'pr', 'rb', 'vb', 'abbreviation'], ['saan', 'pr', 'vb', 'abbreviation'], ['ano', 'dt', 'vb', 'vb', 'cc', 'nn', 'abbreviation']]


In [15]:
initSet = createInitSet(simpDat)
print initSet

{('paano', 'pr', 'vb', 'dt', 'dt', 'nn', 'cc', 'pr', 'abbreviation'): 1, ('ano', 'dt', 'nn', 'vb', 'cc', 'nn', 'pr', 'vb', 'nn', 'abbreviation'): 1, ('saan', 'pr', 'vb', 'abbreviation'): 1, ('saan', 'pr', 'rb', 'vb', 'abbreviation'): 1, ('ano', 'dt', 'vb', 'vb', 'cc', 'nn', 'abbreviation'): 1, ('saan', 'pr', 'rb', 'nn', 'cc', 'nn', 'abbreviation'): 1, ('saan', 'pr', 'cd', 'vb', 'abbreviation'): 1}
